## Connecting with a MongoDB on the cloud using mLab

In [1]:
import pymongo
try:
    #use your database name, user and password here:
    #mongodb://<dbuser>:<dbpassword>@<mlab_url>.mlab.com:57066/<database_name>
    with open("credentials", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
    print([name,password,url,dbname])
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
print(conn)

#Create a database using db = conn.name_db or dictionary access db = conn['name_db']
db = conn[dbname]
print (db)
collection = db.tweets
db.collection_names()

Connected successfully!!!
['alex', 'alex', 'ds145293.mlab.com:45293', 'alex']
MongoClient(host=['ds145293.mlab.com:45293'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['ds145293.mlab.com:45293'], document_class=dict, tz_aware=False, connect=True), 'alex')


['tweets',
 'objectlabs-system',
 'system.indexes',
 'objectlabs-system.admin.collections']

In [2]:
import pandas as pd
df = pd.read_json('./TweetsBarcelona/test.json',orient='index')
df

,createdAt,geoLocation,id,inReplyToUserId,text,userId,userLang,userLocation
tweet1,30-mar-2014 23:51:46,NaN,450389948961943552,728530951,@Weedshine Aquesta pinta millor. O pots fer un...,246452645,ca,"Raval, Pa?sos Catalans"
tweet2,30-mar-2014 23:51:46,"{'latitude': 41.5292058, 'longitude': 1.8507183}",450389949700141056,-1,Jaja = ?,595619618,ca,Catalunya.
tweet3,30-mar-2014 23:51:38,"{'latitude': 41.37657536, 'longitude': 2.16752...",450389916661592064,-1,CADA UNO DA LO QUE TIENE http://t.co/xjSJw7CJf7,248747003,es,Estepa (Sevilla) Espa?a


In [3]:
#collection.insert_many(df.to_dict("records"))

In [4]:
#conn.close()

In [5]:
#db.drop_collection("tweets")

## Twitter api usage

In [6]:
import json
import pymongo
import tweepy

with open('consumer_key', 'r') as f:
    consumer_key =  f.read()
f.closed

with open('consumer_secret', 'r') as f:
    consumer_secret = f.read()
f.closed

with open('access_key', 'r') as f:
    access_key = f.read()
f.closed

with open('access_secret', 'r') as f:
     access_secret = f.read()
f.closed


#Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#Do something
USER_NAME = "usuarialatzar"
user = api.get_user(id=USER_NAME)
print (user.name)

PSKS


In [10]:
key_words = [
    'party','fiesta','fest ','fest!','festival','festa',
    'club','disco','discoteca','vip','dj',
    'night','noche','nit','nocturna',
    'event','show',
    'cine','teatr','theatre',
    'concert','concierto',
    'pub','bar',
    'beer','drink','drunk','cervesa','cerveza','mojito','gintonic','cocktail','wine',
    'whiskey','rum','wine','sangria',
    'copas','copes','tapas','tapes',
    'dance','dancing','ball','baile','baila','salsa',
    'musical','rave ',
    'going out','go out','hang out','goingout','goout','hangout',
    'having fun','havingfun','celebra'
]
false_key_words = [
    'barcelon','bare ','baron',
    'travel',
    'public',
    'manifest',
    'Trump','rumia','forum',
    'peniten',
    'brave',
    'bonit','dignit'
] #bar in barcelona, rave in travel

def criteria(jdata):
    return geo_check(jdata) and key_word_check(jdata['text'])
def geo_check(jdata):
    if jdata['coordinates'] is not None:
         print('coords',jdata['coordinates']['coordinates'],'geo',jdata['geo'])
    return jdata['coordinates'] is not None or jdata['geo'] is not None

def key_word_check(text):
    import re
    print(text)
    text = text.lower()
    big_regex = re.compile('|'.join(map(re.escape, false_key_words)))
    text = big_regex.sub("", text)
    for key_word in key_words:
        if key_word in text:
            print('Stored! Keyword ', key_word, ' found')
            return True
    return False

In [ ]:
from tweepy import Stream,StreamListener

class listener(StreamListener):
    def __init__(self):
        super(StreamListener, self).__init__()
        self.num_tweets = 0
        self.db = db
        self.collection = self.db.tweets
    
    def on_data(self, status):
        if self.num_tweets < 1e4:
            jdata = json.loads(status)
            if criteria(jdata) is True:
                document={'id':jdata['id'],'geo':jdata['geo'],'coordinates':jdata['coordinates'],'text':jdata["text"], 'created':jdata["created_at"]}
                self.collection.insert_one(document) 
                print (jdata['geo'],jdata["text"])
                self.num_tweets += 1
            return True
        else:
            return False
    
    def on_error(self, status):
        print (status)

# Catch all tweets in Barcelona area and print them
twitterStream = Stream(auth, listener())
twitterStream.filter(locations=[2.0504377635,41.2787636541,2.3045074059,41.4725622346])
print ("done")

coords [2.13693202, 41.38947107] geo {'type': 'Point', 'coordinates': [41.38947107, 2.13693202]}
Esperando la actuación de la banda LEMOZINE (@ Bikini in Barcelona) https://t.co/fZ1NZ7rSIW
coords [2.15982, 41.3863] geo {'type': 'Point', 'coordinates': [41.3863, 2.15982]}
Drinking a Grande Noirceur by @Brasseries_DDC at @biercab — https://t.co/d9PXmqMwi0 #photo
Stored! Keyword  drink  found
{'type': 'Point', 'coordinates': [41.3863, 2.15982]} Drinking a Grande Noirceur by @Brasseries_DDC at @biercab — https://t.co/d9PXmqMwi0 #photo
coords [2.15571396, 41.39113506] geo {'type': 'Point', 'coordinates': [41.39113506, 2.15571396]}
Avui toca #trinxat (@ La Camarga in Barcelona w/ @djkram) https://t.co/mSoxa4hP2s
Stored! Keyword  dj  found
{'type': 'Point', 'coordinates': [41.39113506, 2.15571396]} Avui toca #trinxat (@ La Camarga in Barcelona w/ @djkram) https://t.co/mSoxa4hP2s
coords [2.18333, 41.3833] geo {'type': 'Point', 'coordinates': [41.3833, 2.18333]}
Labios rosa y alma inquieta.
De 

coords [2.17492819, 41.3875151] geo {'type': 'Point', 'coordinates': [41.3875151, 2.17492819]}
#GSTHEQUEEN2017 en Palau de la Música Catalana https://t.co/YAiEGBE1Ch
coords [2.15160308, 41.37135396] geo {'type': 'Point', 'coordinates': [41.37135396, 2.15160308]}
Litografía 50x70 , númerada i dedicada a la nostre ong. 65€. Autor A. Mirasierras. #nadal… https://t.co/9y69ut7yc6


In [93]:
collection.count()
print (collection.count())
for doc in collection.find():
    print (criteria(doc))

141
@popthatpartybcn PRES: 
Este Miércoles 8 de noviembre - @Asap__tyy -
Show en directo en… https://t.co/uJF7HLKv6c
Stored! Keyword  party  found
True
¡Os presento a mi club!
Quizá no las conozcáis a todas pero os puedo… https://t.co/BQg3zfNx5r
Stored! Keyword  club  found
True
Last night in Barcelona 😢 #srprsme #boomerang @ El National, Barcelona https://t.co/y1hntIvc5K
Stored! Keyword  night  found
True
@popthatpartybcn PRES: 
Este Miércoles 8 de noviembre - @Asap__tyy -
Show en directo en… https://t.co/uJF7HM26uM
Stored! Keyword  party  found
True
"Freedom for Political Prisoners"
An exciting walk home tonight in #barcelona @ Plaça de Catalunya https://t.co/PSBB9unA70
Stored! Keyword  night  found
True
Republic day 7

#freecatalanpoliticalprisoners @ Parlament De Catalunya https://t.co/Cr7LGV7Dbz
False
#Escorts esta noche  en #Barcelona: Amanda, Angie, Chantal, Desire, Jasmine, Katherine, Kattya, Laila, Lili, Lucía https://t.co/SFDeDTXDS9
Stored! Keyword  noche  found
True
#barcelo

Nit màgica al Palau! Llibertat presos polítics! No esteu sols! Gràcies @dracs1991… https://t.co/yk5D1n76vs
Stored! Keyword  nit  found
True


In [108]:
import json

# Make it work for Python 2+3 and with Unicode
import io

with io.open('./TweetsBarcelona/2017/tweets_2017.json', 'w', encoding='utf8') as outfile:
    for doc in collection.find():
        str_=str(doc)
        outfile.write(str_+'\n')